In [ ]:
from dotenv import load_dotenv
load_dotenv()
import ipywidgets as widgets
from ipywidgets import HBox, VBox, HTML
from IPython.display import display
from giga.app.config import get_registered_countries, get_country_defaults
from giga.viz.notebooks.components.html.sections import section
from giga.viz.notebooks.parameters.groups.scenario_parameter_manager import (
    ScenarioParameterManager,
)
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput


option = widgets.RadioButtons(
    options=['Update an existing country', 'Add a new country'],
    disabled=False,
    index=None
)

country_select = widgets.Dropdown(
    options=get_registered_countries(),
    description='Country:',
)

country_input = widgets.Text(
    description='Country:',
)

# Set up a custom scenario parameter manager only includes
# default params (eg does not include scenario selection)
scenario_manager = ScenarioParameterManager(base_parameters=[])


default_params = CostEstimationParameterInput(
    scenario_parameter_manager=scenario_manager,
    local_data_workspace='workspace',
    show_map=False,
    school_selection=None,
    show_defaults=False)
# TODO use country_name_to_key
output = widgets.Output()
country_config_output = widgets.Output()
optional_info_output = widgets.Output()
save_output = widgets.Output()
full_config_output = widgets.Output()
fetching_data_output = widgets.Output()

first_time = True
def on_selected_country_changed(change):
    # Show the appropriate country widget depending on operation.
    output.clear_output()
    with output:
        if creating_new_country():
            display(country_input)
        else:
            display(country_select)
            # Fetch data for the default-selected country immediately
            fetch_data()

def fetch_data():
    global first_time
    fetching_data_output.clear_output()
    country_name = get_selected_country()
    with fetching_data_output:
        # Fetch current country configuration from object store.
        display(section(
            title="",
            contents=HTML("Updating country information...")
        ))
        all_defaults = get_country_defaults()
        assert country_name in all_defaults, "Could not find configuration for existing country!"
        
        # Update the parameter sheet with the current configuration
        config = all_defaults[country_name]
        default_params.set_defaults_for_country(country_name, config)
    fetching_data_output.clear_output()
    if not first_time:
        return
    first_time = False
    with full_config_output:
        display(country_config_output)
        display(optional_info_output)
        display(save_output)


# Update the page when the selected country changes
option.observe(on_selected_country_changed, 'value')
country_select.observe(on_selected_country_changed, 'value')

# Display the option
display(section(
    title="Update Countries",
    contents=VBox([
        option,
        output
    ]),
    extra_class="header"))
display(fetching_data_output, full_config_output)

def creating_new_country():
    return option.value == 'Add a new country'

def get_selected_country():
    if not creating_new_country():
        if country_select.value is not None:
            return country_select.value
    elif country_input.value.strip() != "":
        return country_input.value.strip().replace(" ", "_").lower()
    return None

In [ ]:
import json


from giga.utils.globals import COUNTRY_DEFAULT_RELATIVE_DIR
from giga.data.space.country_updater import CountryUpdateRequest
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload
from giga.viz.notebooks.components.html.sections import section


giga_country_code = widgets.Text()

country_center_lat = widgets.FloatText()
country_center_long = widgets.FloatText()

def up_btn(description, accept=".csv", **kwargs):
    layout = widgets.Layout(width='300px')
    return widgets.FileUpload(accept=accept, multiple=False, 
                              description=description, layout=layout, 
                              max_file_size=100,  # MB
                              **kwargs)

# Setting up country defaults

defaults_upload_output = widgets.Output()
upload_defaults = up_btn(accept='.json', description="Upload country config (.json)")

def on_upload_change(change):
    try:
        imported = json.loads(upload_defaults.value[0].content.tobytes())
    except IndexError:
        imported = {}
    defaults_upload_output.clear_output()
    default_params.update(imported)
upload_defaults.observe(on_upload_change, names='value')

country_defaults = up_btn(accept='.csv', description="Country defaults (.csv)")
# display(country_defaults)


with country_config_output:
    display(section(
        title="Country Configuration",
        contents=VBox([
            upload_defaults,
            HTML('<br/>'),
            HBox([
                section(
                    title="Country Info",
                    contents=VBox([
                        HBox([
                            HTML("Giga country code:"),
                            giga_country_code
                        ]),
                        HBox([
                            HTML("Country center:"),
                            VBox([
                                HBox([HTML("Lat:"), country_center_lat]),
                                HBox([HTML("Long:"), country_center_long])
                            ])
                        ]),
                    ])
                ),
                section(
                    title="Scenario Defaults",
                    contents=default_params.scenario_parameter_input()
                )
            ])
        ]),
        extra_class="center"
    ))
    display(default_params.country_default_parameter_input())

In [ ]:
cellular = up_btn(accept='.csv', description="Cell tower locations (.csv)")
fiber = up_btn(accept='.csv', description="Fiber node locations (.csv)")
schools_supplemental = up_btn(accept='.csv', description="Supplemental school info (.csv)")

upload_output = widgets.Output()
upload_button = widgets.Button(description="Upload")

with optional_info_output:
    display(section(
        title="Optional information",
        contents=VBox([
            cellular,
            fiber,
            schools_supplemental,
        ]),
        extra_class="center"
    ))

with save_output:
    display(section(
        title="Save",
        contents=VBox([
            upload_button,
            upload_output
        ]),
        extra_class="footer"
    ))

def on_upload_change(change):
    upload_output.clear_output()
    with upload_output:
        req = CountryUpdateRequest()
        req.country_name = get_selected_country()
        req.country_defaults = country_defaults
        req.cellular = cellular
        req.fiber = fiber
        req.schools_supplemental = schools_supplemental
        req.attempt()
        

upload_button.on_click(on_upload_change)